<a href="https://colab.research.google.com/github/nada912/Neural-Networks/blob/main/Heart_Disease_Predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from google.colab import files

#**Upload files into Google Colab Environment**



In [2]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving test.csv to test.csv
Saving train.csv to train.csv
User uploaded file "test.csv" with length 7447 bytes
User uploaded file "train.csv" with length 39015 bytes


#**Load the data from the uploaded files**



In [3]:
import pandas as pd

file_name = "train.csv"
train = pd.read_csv(file_name)

file_name1 = "test.csv"
test = pd.read_csv(file_name1)

#**Explore data**

In [4]:
train.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
2,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
3,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
4,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0


In [17]:
test.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat
1,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat
2,39,M,ATA,120,204,0,Normal,145,N,0.0,Up
3,41,M,ASY,110,289,0,Normal,170,N,0.0,Flat
4,52,M,ASY,120,182,0,Normal,150,N,0.0,Flat


#**Define the target variable and features**



In [6]:
X = train.drop("HeartDisease", axis=1)
y = train["HeartDisease"]

Perform one-hot encoding for all categorical variables

In [7]:
X_train_enc = pd.get_dummies(X, columns=["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"])
X_test_enc = pd.get_dummies(test, columns=["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"])

#**Split the data into training and testing sets**

Standardize the features in the training and testing data

In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_enc)
X_test_scaled = scaler.transform(X_test_enc)

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_train_scaled, y, test_size=0.2, random_state=42)

#**Choose ML model**

##**Complex Sequential Model**

In [12]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers

In [13]:
model = keras.Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

Learning Rate Scheduling

In [14]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler

def lr_scheduler(epoch, lr):
    if epoch % 5 == 0:  # Adjust the schedule as needed
        return lr * 0.9
    return lr

optimizer = Adam(learning_rate=0.001) # best lr = 0.001
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

lr_callback = LearningRateScheduler(lr_scheduler)

Monitor the validation loss

In [15]:
from tensorflow.keras.callbacks import EarlyStopping

# Without improvement after 5 consecutive epochs, training will stop early to prevent overfitting.
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

Train the model

In [16]:
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
20/20 [==============================] - 2s 33ms/step - loss: 0.7960 - accuracy: 0.5944 - val_loss: 0.6214 - val_accuracy: 0.8301
Epoch 2/10
20/20 [==============================] - 0s 5ms/step - loss: 0.6301 - accuracy: 0.7718 - val_loss: 0.5203 - val_accuracy: 0.8497
Epoch 3/10
20/20 [==============================] - 0s 4ms/step - loss: 0.5484 - accuracy: 0.8259 - val_loss: 0.4783 - val_accuracy: 0.8431
Epoch 4/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4902 - accuracy: 0.8424 - val_loss: 0.4626 - val_accuracy: 0.8431
Epoch 5/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4722 - accuracy: 0.8654 - val_loss: 0.4562 - val_accuracy: 0.8366
Epoch 6/10
20/20 [==============================] - 0s 4ms/step - loss: 0.4445 - accuracy: 0.8686 - val_loss: 0.4561 - val_accuracy: 0.8366
Epoch 7/10
20/20 [==============================] - 0s 15ms/step - loss: 0.4232 - accuracy: 0.8670 - val_loss: 0.4565 - val_accuracy: 0.8497
Epoch 8/10
20/20 [

Make predictions on the test data

In [18]:
y_pred = model.predict(X_test_scaled)
y_pred_binary = (y_pred > 0.5).astype(int)
y_pred_binary = y_pred_binary.ravel()

5/5 [==============================] - 0s 5ms/step


##**Random Forest Classifier**

Create the model

In [12]:
from sklearn.ensemble import RandomForestClassifier

In [13]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=100)

Train the model

In [14]:
rf_classifier.fit(X_train_scaled, y)

RandomForestClassifier(random_state=100)

Make predictions on the test data

In [15]:
y_pred_binary = rf_classifier.predict(X_test_scaled)

#**Create submission file**

Create a Dataframe with the desired columns

In [18]:
output = pd.DataFrame({'id': range(1, len(y_pred_binary) + 1), 'HeartDisease': y_pred_binary})

Save Dataframe as csv file

In [19]:
output.to_csv('submission.csv', index=False)
print("Your file was successfully saved!")

Your file was successfully saved!


Download file to my local machine

In [21]:
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>